In [ ]:
# Initial setup
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from numpy.polynomial.polynomial import polyfit
!pip install adjustText
from adjustText import adjust_text

# Import data
# 2019 survey
q_19 = pd.read_csv("../input/kaggle-survey-2019/questions_only.csv")
mc_19 = pd.read_csv("../input/kaggle-survey-2019/multiple_choice_responses.csv", skiprows = range(1,2))
text_19 = pd.read_csv("../input/kaggle-survey-2019/other_text_responses.csv")
schema_19 = pd.read_csv("../input/kaggle-survey-2019/survey_schema.csv")

# 2018 survey
mc_18 = pd.read_csv("../input/kaggle-survey-2018/multipleChoiceResponses.csv", skiprows = range(1,2), low_memory = False)

# 2017 survey
mc_17 = pd.read_csv("../input/kaggle-survey-2017/multipleChoiceResponses.csv", encoding = "ISO-8859-1", low_memory = False)

# World Bank Data
wdi = pd.read_csv("../input/2019-world-development-indicator-wdi/WDIData.csv")
wdi.columns = [col.lower().replace(" ", "_") for col in wdi.columns]
wdi_country = pd.read_csv("../input/2019-world-development-indicator-wdi/WDICountry.csv")
wdi_country.columns = [col.lower().replace(" ", "_") for col in wdi_country.columns]

# Color coding for income group
color_dict = dict({'High income': '#293462', 
                      'Upper middle income' : '#32dbc6',
                      'Lower middle income': '#b7e778'})
# Legend markers
markers = [plt.Line2D([0,0],[0,0],color=color, marker='o', linestyle='') for color in color_dict.values()]
# Set plt parameters
plt.rc('axes', axisbelow=True)
plt.rc('xtick', labelsize=12) 
plt.rc('ytick', labelsize=12)
plt.rc('grid', alpha=0.3)

# 1. Executive Summary

With a focus on the <span style="background-color: #ffe7ad"> data science community in developing countries</span>, this notebook aims to explore if the community in developing countries differs from that in developed countries. By analyzing 2019 Kaggle Machine Learning & Data Science survey data, this notebook compares a) the demographic composition of data science practitioners and b) the popularity of different programming languages in developing countries with those in developed countries. Here are the main takeaways from this data story:

- __Representation:__ According to the World Bank, developing countries are defined as middle- and low-income countries based on Gross National Income (GNI) per capita. For the 2019 Kaggle ML & DS Survey, over half of the survey respondents live in middle-income countries, and no low-income country is represented in the dataset.
- __Age & Gender:__ Overall, Kaggle survey respondents living in developing countries tend to be younger than those living in developed countries, as there are proportionally more respondents under 30 years old living in developing countries. Controlling for age, the gender ratio in developing countries is more skewed than that in developed countries.
- __Occupation:__ Among all survey respondents, there are proportionally more students in developing countries and proportionally more data science professionals in developed countries.
- __Programming Languages:__ The top 3 most popular programming languages in both developing and developed countries are Python, SQL, and R. However, there are proportionally more Python, SQL and R users in higher-income countries, and proportionally more C and C++ users in lower-income countries.

In summary, this notebook suggests that <span style="background-color: #ffe7ad"> Kagglers living in developing countries are more likely to be data science learners at younger ages </span>. I hope that this data story can help Kaggle better understand the learning needs and priorities of users in developing countries and make data science accessible to a wider range of audiences. 

---

# 2. Introduction

Kaggle is an online community that connects and empowers data science enthusiasts around the world. It democratizes data science by allowing users to publish and explore data sets free of charge as well as to participate in data science competitions hosted by big-name companies and organizations. As the platform has attracted over 2.5MM users globally, it provides us with a valuable window on the worldwide data science community [1]. 

This notebook focuses on a specific subset of the data science community: <span style="background-color: #ffe7ad"> data science practitioners in developing countries</span>. It aims to explore how the data science community in developing countries may differ from that in developed countries by analyzing Kaggle survey data. Specifically, this notebook will examine two aspects:

- <span style="background-color: #ffe7ad"> The demographic composition</span> of data science pratitioners living in developing countries
- <span style="background-color: #ffe7ad"> The popularity of different programming languages</span> used by data science pratitioners living in developing countries

---

# 3. Data
## 3.1 Data Sources

This notebook uses data from two sources: Kaggle survey data and World Development Indicator data released by the World Bank. 

- __Kaggle survey data__: Kaggle recently conducted its third annual Machine Learning and Data Science Survey, which received 19,717 usable respondents from 171 countries and regions. The survey includes two categories of questions: a) demographics and b) data science related questions. The first category includes questions about respondents’ gender, age, education level, country of residence, etc. The second category includes questions about how respondents learn data science and what technologies respondents use frequently at work. 
- __World Bank World Development Indicator (WDI)__: WDI is a collection of internationally comparable statistics about global development with around 1,600 time series indicators for 217 economies [2]. It provides us with important metrics that qualify the level of development and economic strength of different countries and regions. 

Although the ultimate goal of this analysis is to study the data science community in developing countries by analyzing Kaggle survey data, we need to acknowledge that the dataset will be biased in two ways. 

- Kaggle users in developing countries will be a biased representation of the broader data science community in developing countries. The 2019 Kaggle survey data only includes 26 middle income countries and no low income country. In addition, as Kaggle currently is only available in English, and the official languages in most of the developing countries are not English, Kaggle users only represent the English speakers, who may be younger or may have higher education levels than an average data science practitioner in developing countries.
- Kaggle survey respondents may be a biased representation of Kaggle users because of self-selection bias.

## 3.2 Definition of Developing Countries

In this notebook, <span style="background-color: #ffe7ad"> developing countries are defined as middle- or low-income countries </span>. The World Bank categorizes countries into four groups (high income, upper middle income, lower middle income and low income) based on Gross National Income (GNI) per capita [3]. GNI is an alternative to Gross Domestic Product (GDP), and it can be a more accurate indicator of the residents’ income level in some countries. Please see here to learn about how the World Bank classifies countries and regions into income groups [3]. 

In [ ]:
# Add legend
plt.figure(figsize=(15,0.1))
plt.legend(markers, color_dict.keys(), prop={'size': 12}, 
           bbox_to_anchor=(0.8, -0.3), ncol = 3)
plt.axis("off")
plt.show()

# Create a map
import plotly.graph_objects as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True) 
color_scale = [[0.0,'rgb(41,52,98)'], [0.5,'rgb(50, 219, 198)'], [1.0, 'rgb(183, 231, 120)']]
fig = go.Figure(data=go.Choropleth(
        locations = wdi_country["country_code"],
        z = np.where(wdi_country["income_group"] == "High income", 0, 
                          np.where(wdi_country["income_group"] == "Upper middle income", 0.5, 1)),
        text = [str(wdi_country.loc[i, "short_name"]) + ": " + str(wdi_country.loc[i, "income_group"]) for i in range(len(wdi_country))],
        colorscale = color_scale,
        showscale = False))

fig.update_layout(
        geo_scope="world",
        title = go.layout.Title(text = 'Countries and Regions by Income Group', 
                                font_size = 15, x=0.5),
    )
fig.show()

## 3.3 Data Overview

Because the World Development Indicators (WDI) data is used to classify countries or regions into income groups, this analysis only focuses on countries or regions that are present in the WDI dataset. As a result, although there are 58 unique country or region labels, including “others”, reported in Kaggle Survey data, only 55 of them are considered. 

### Summary Statistics by Income Group
Out of 55 countries and regions in Kaggle survey data, 26 are middle-income countries, and <span style="background-color: #ffe7ad"> no low-income country is represented </span>. Among 18,107 respondents from these 55 countries, 46.8% of them live in developed countries or high-income countries, and <span style="background-color: #ffe7ad"> 53.2% of respondents live in developing countries or middle-income countries </span>. 

In [ ]:
# Compute number of respondent by country
def res_by_country(df, col_name):
    """
    Compute number of survey respondents by country
    Input:
    df:         multiple choice response dataframe
    col_name :  column name for country
    
    Output:
    res:        dataframe with the number of respondents by country
    """
    # Combine country name for Korea
    df[col_name] = np.where(df[col_name] == "Republic of Korea", "South Korea", df[col_name])
    # Compute number of respondents per country
    res = pd.DataFrame(df.groupby(col_name)[df.columns[0]].count()).reset_index()
    res.columns = ["country/region", "respondent_count"]
    res = res.sort_values("respondent_count", ascending = False).reset_index(drop = True)
    return res

# Compute number of respondents for 2017-2019 surveys
res_19 = res_by_country(mc_19, "Q3")
res_18 = res_by_country(mc_18, "Q3")
res_17 = res_by_country(mc_17, "Country")

# Identify country code for each country/region
country_code = res_19[["country/region", "respondent_count"]] \
                .merge(wdi_country[["table_name", "country_code"]], 
                       how = "left", left_on = "country/region", right_on = "table_name")
# Inpute country code
country_code.iloc[1, 3] = "USA"
country_code.iloc[5, 3] = "RUS"
country_code.iloc[8, 3] = "GBR"
country_code.iloc[13, 3] = "TWN"
country_code.iloc[21, 3] = "KOR"
country_code.iloc[27, 3] = "VNM"
country_code.iloc[30, 3] = "EGY"
country_code.iloc[37, 3] = "IRN"
country_code.iloc[48, 3] = "HKG"

# Identify population for each country/region
ppl_df = wdi[wdi["indicator_name"] == "Population, total"][["country_code", "2018"]]
ppl_df.columns = ["country_code", "total_ppl"]

# Find GDP per capita (constant 2010 US$)
gni_df = wdi[wdi["indicator_name"] == "GNI per capita (constant 2010 US$)"][["country_code", "2017"]]
gni_df.columns = ["country_code", "gni_per_capita"]

# Clean the table and join population df
country_code = country_code[["country/region", "country_code"]]\
                        .merge(wdi_country[["country_code", "income_group", "short_name"]]) \
                        .merge(ppl_df) \
                        .merge(gni_df)

# Join world bank data
res_19 = res_19.merge(country_code)
res_18 = res_18.merge(country_code)
res_17 = res_17.merge(country_code)

# Show summary stat for 2019 Survey
stat_19 = res_19.groupby("income_group").agg({"country/region": "count",
                                                "respondent_count": "sum"})
stat_19.index.name = "Income Group"
stat_19.columns = ["Number of Countries/Regions", "Respondent Count"]
stat_19["% Total Respondent"] = [str(i) + "%" for i in round(stat_19["Respondent Count"] * 100 / np.sum(stat_19["Respondent Count"]), 2)]
stat_19.loc["All countries/region", :] = [55, np.sum(stat_19["Respondent Count"]), "100%"]
stat_19

### Breakdown by Income Group

The number of survey respondents doubled from 2017 to 2018, as the number of Kaggle users increased significantly after Kaggle was acquired by Google in 2017. Regardless of the number of survey respondents, all of these surveys show that <span style="background-color: #ffe7ad">around half of the respondents live in developing countries.</span> 

In [ ]:
# Compute percent of respondent by income group
res_by_income = pd.DataFrame({"2017": res_17.groupby("income_group")["respondent_count"].sum(),
                             "2018": res_18.groupby("income_group")["respondent_count"].sum(),
                             "2019": res_19.groupby("income_group")["respondent_count"].sum()}).T
res_by_income = res_by_income.reset_index()
res_by_income.columns = ["year", 'High income', 'Lower middle income', 'Upper middle income']
res_by_income["respondent_count"] = [res_17["respondent_count"].sum(), 
                                    res_18["respondent_count"].sum(),
                                    res_19["respondent_count"].sum()]

# Create a stacked bar chart
plt.figure(figsize=(15,4))
plt.subplot(121)
plt.grid()
plt.bar(x = res_by_income["year"],
        height = res_by_income["Lower middle income"],
        color = color_dict["Lower middle income"])
plt.bar(x = res_by_income["year"],
        height = res_by_income["Upper middle income"],
        color = color_dict["Upper middle income"],
        bottom = res_by_income["Lower middle income"])
plt.bar(x = res_by_income["year"],
        height = res_by_income["High income"],
        color = color_dict["High income"],
        bottom = res_by_income["Upper middle income"] + res_by_income["Lower middle income"])
plt.ylabel("Number of Respondents", size = 12)
plt.xlabel("Year", size = 12)
plt.title("Number of Respondent by Income Group (2017-2019)", size = 14)


# Create line chart for percent of respondent by income group by year
plt.subplot(122)
plt.grid()
for income_group in color_dict.keys():
    plt.scatter(res_by_income["year"], 
             res_by_income[income_group] * 100 / res_by_income["respondent_count"], 
             color = color_dict[income_group], edgecolors = 'grey')
    plt.plot(res_by_income["year"], 
             res_by_income[income_group] * 100 / res_by_income["respondent_count"], 
             color = color_dict[income_group], label = res_by_income,
             ls = 'dotted', alpha = 0.5)
plt.ylabel("% of Respondents", size = 12)
plt.xlabel("Year", size = 12)
plt.title("Percent of Respondent by Income Group (2017-2019)", size = 14)
plt.ylim(0, 60)
 
# Add legend 
plt.legend(markers, color_dict.keys(), prop={'size': 12}, 
           bbox_to_anchor=(0.5, -0.2), ncol = 3)
plt.show()

### Breakdown by Country of Residence

The following histogram visualizes the number of respondents living countries and regions with more than 100 respondents. Only two countries, India (lower middle income) and the United States (high income) have over 3,000 respondents. These two countries alone account for around 40% of survey respondents in 2019. 

Again, it is worthy to point out that the histogram here may not ba an accurate representation of the data science communities in different countries and regions in terms of their relative size. For example, there are over 3,000 respondents living in India and the United States, two of the most populated countries in the world. However, only 570 respondents report that they live in China, where data science is a rapidly developing field but the English speaking population is much smaller.

In [ ]:
# Filter to a subset of countries/regions w/ respondent > 100
country_filter = (res_19["country/region"] != "Other") & (res_19["respondent_count"] > 100)

# Create the bar chart
plt.figure(figsize=(15,4))
plt.bar(x = "country_code",
        height = "respondent_count",
        color = np.array([color_dict[i] for i in res_19["income_group"]]),
        data = res_19[country_filter])
plt.grid(alpha = 0.3)
plt.ylabel("Number of Respondents", size = 12)
plt.xlabel("Country/Region", size = 12)
plt.title("Number of Respondents by Country (2019)", size = 14)
plt.legend(markers, color_dict.keys(), prop={'size': 12})
plt.xticks(rotation = 45)
plt.show()

---

# 4. Demographic Composition
### Age

The first visualization on the left shows the percentage of respondents by age group and income group. As shown in the visualization, <span style="background-color: #ffe7ad"> younger respondents make up a higher percentage of overall respondents in developing (middle-income) countries than in developed (high-income) countries </span>. Specifically, respondents younger than 24 years old account for over 50% of total respondents in lower middle income countries but only 15% in high income countries. 

### Gender

The box plot on the right illustrates the distribution of % of female respondents by income group. We may expect developing countries, where the gender gap tends to be larger, will have fewer female respondents than high income countries. However, interestingly and strikingly at the same time, <span style="background-color: #ffe7ad"> the gender gap observed from the data is equally large across three income groups: in different countries and regions included in the dataset, only 5.1% to 25.4% of respondents are female, and the median % of female respondents ranges from 13% to 17% for all income groups. </span>

In [ ]:
# Demographic composition
# Gender
gender_19 = mc_19.groupby(["Q3", "Q2"])["Q1"].count().reset_index() \
                .merge(country_code, left_on = "Q3", right_on = "country/region") \
                .merge(res_19[country_filter][["country/region", "respondent_count"]]) \
                .drop(["Q3"], axis = 1)
gender_19.columns = ["gender", "respondent_by_gender"] + list(gender_19.columns[2:])
# Compute perc
gender_19["perc_respondent"] = gender_19["respondent_by_gender"] * 100 / gender_19["respondent_count"]

# Age
age_19 = mc_19[["Q1", "Q3"]].merge(country_code, left_on = "Q3", right_on = "country/region")\
            .groupby(["income_group", "Q1"])["Q3"].count()\
            .reset_index()
age_19.columns = ["income_group", "age_group", "respondent_by_age"]

# Create line chart for age group
plt.figure(figsize=(15,4))
plt.subplot(121)
plt.grid(alpha = 0.3)
for income_group in color_dict.keys():
    by_income = age_19[age_19["income_group"] == income_group]
    plt.scatter(by_income["age_group"],
                by_income["respondent_by_age"] * 100 / by_income["respondent_by_age"].sum(),
                color = color_dict[income_group], edgecolors = 'grey')
    plt.plot(by_income["age_group"],
             by_income["respondent_by_age"] * 100 / by_income["respondent_by_age"].sum(),
             color = color_dict[income_group], ls = 'dotted', alpha = 0.5)
plt.xlabel("Age Group", size = 12)
plt.xticks(size = 11)
plt.yticks(size = 11)
plt.ylabel("% of Total Respondents", size = 12)
plt.title("% of Respondents by Age Group (2019)")
plt.ylim(0, 35) 

# Create box plot for % female
plt.subplot(122)                      
sns.boxplot(x =  "income_group", y = "perc_respondent", 
            data = gender_19[(gender_19["gender"] == "Female")],
            palette = color_dict,
            order = color_dict.keys())
plt.grid(alpha = 0.3)
# Add text annotation
plt.text(1.6, 7, "Bangladesh (5.1%)", size = 11)
plt.text(1.7, 27, "Kenya (25.4%)", size = 11)
plt.text(-0.2, 5, "Japan (8.0%)", size = 11)
plt.text(-0.3, 24, "Canada (22.0%)", size = 11)
plt.text(0.7, 25, "Turkey (23.0%)", size = 11)
# Set axis parameters
plt.xlabel("Income Group", size = 12)
plt.xticks(size = 11)
plt.yticks(size = 11)
plt.ylabel("% of Total Respondents", size = 12)
plt.title("% of Respondents = Female (2019)")
plt.ylim(0, 35)

# Add legend
plt.legend(markers, color_dict.keys(), prop={'size': 12}, 
           bbox_to_anchor=(0.5, -0.2), ncol = 3)
plt.show()

Does it mean that the broader data science communities in all countries have similarly sized gender gap regardless of the development level? My answer is it depends on how you measure the gender gap. Looking at the boxplot alone, three income groups seem to display similar distribution in terms of % of female respondents. However, if we break down the number by age group, we can observe two interesting patterns:
- __Comparision across age group__: There are proportionally more female respondents from the younger age groups than the older age group regardless of the income level. 
- __Comparison within the same age group__: there are proportionally more female respondents in developed (high income) countries than in developing (non-high income) countries, except for the 40-44 group. 

Combined with the fact that there is a higher % of younger respondents in developing countries, these two findings explain why the overall % of female respondents are similar across income groups and suggest that <span style="background-color: #ffe7ad"> developing countries may have wider gender gap when age is taken into consideration. </span> 

In [ ]:
# Investigate number of respondents by gender, age and income group
gender_age_19 = mc_19[["Q1", "Q2", "Q3"]]
gender_age_19.columns = ["age_group", "gender", "country/region"]
gender_age_19 = gender_age_19.merge(country_code) \
                            .groupby(["income_group", "age_group", "gender"])["country_code"]\
                            .count().reset_index() \
                            .merge(age_19)                     
gender_age_19.columns = list(gender_age_19.columns[:3]) + ["respondent_count", "total_respondent"]

# Count total number of respondents by income and age
gender_age_all = gender_age_19.groupby(["age_group", "gender"])["respondent_count"].sum().reset_index()
age_all = gender_age_all.groupby(["age_group"])["respondent_count"].sum().reset_index()
age_all.columns = ["age_group", "total_respondent"]

# Compute percent of respondents = female
gender_age_19 = gender_age_19[gender_age_19["gender"] == "Female"]
gender_age_19["perc_female"] = gender_age_19["respondent_count"] * 100 / gender_age_19["total_respondent"]

gender_age_all = gender_age_all.merge(age_all)
gender_age_all = gender_age_all[gender_age_all["gender"] == "Female"]
gender_age_all["perc_female"] = gender_age_all["respondent_count"] * 100 / gender_age_all["total_respondent"]

plt.figure(figsize=(15,4))
bar_width = 0.2
r1 = np.arange(6)
r2 = [x - bar_width for x in r1]
r3 = [x + bar_width for x in r1]
r4 = [x + bar_width for x in r3]
ind_list = [r1, r3, r4]

# Create grouped bar chart
plt.grid(alpha = 0.3)
plt.bar(r2, gender_age_all["perc_female"][:6], width = bar_width, color = "grey")
for i in range(len(ind_list)):
    income_group = list(color_dict.keys())[i]
    data = gender_age_19[gender_age_19["income_group"] == income_group]
    plt.bar(ind_list[i], data["perc_female"][:6], width = bar_width, color = color_dict[income_group])
    
plt.xticks([r for r in range(6)], 
           gender_age_all["age_group"][:6], size = 11)
plt.yticks(size = 11)
plt.ylim(0, 25)
plt.xlabel("Age Group", size = 12)
plt.ylabel("% of Respondents in Age Group", size = 12)
plt.title("% of Respondents = Female by Age Group (2019)")

# Add legend
temp_markers = [plt.Line2D([0,0],[0,0],color=color, marker='o', linestyle='') for color in ["grey"] + list(color_dict.values())]
plt.legend(temp_markers, ["All countries/regions"] + list(color_dict.keys()), prop={'size': 12}, 
           bbox_to_anchor=(0.9, -0.2), ncol = 4)
plt.show()

### Occupation

The following grouped histogram visualizes the % of respondents by occupation. Because there is a higher percentage of younger respondents in developing countries, the histogram shows <span style="background-color: #ffe7ad"> there are proportionally more students in developing countries than in developed countries as expected </span>. In addition, developing countries under-index “Data Scientist” and slightly over-index “Software Engineer”.

In [ ]:
# Calculate the number of respondent by income, occupation group
occ_19 = mc_19[["Q3", "Q5"]].merge(country_code[["country/region", "income_group"]], 
                                      left_on = "Q3", right_on = "country/region") \
                            .groupby(["income_group", "Q5"])["Q3"].count() \
                            .reset_index()
occ_19.columns = ["income_group", "occupation", "respondent_by_occ"]

# Compute percent of respondent by income, occupation group
total_occ_19 = occ_19.groupby("income_group")["respondent_by_occ"].sum().reset_index()
total_occ_19.columns = ["income_group", "total_respondent"]
occ_19 = occ_19.merge(total_occ_19)
occ_19["perc_respondent"] = occ_19["respondent_by_occ"] * 100 / occ_19["total_respondent"]

# Compute percent of respondent by occupation group across income group
total_occ_19 = occ_19.groupby("occupation")["respondent_by_occ"].sum().reset_index()
total_occ_19["total_perc_respondent"] = total_occ_19["respondent_by_occ"] * 100 / total_occ_19["respondent_by_occ"].sum()
occ_19 = occ_19.merge(total_occ_19[["occupation", "total_perc_respondent"]]) \
                .sort_values("total_perc_respondent", ascending = False)

# Create grouped histogram
plt.figure(figsize=(15,4))
bar_width = 0.2
r1 = np.arange(len(total_occ_19[:8]))
r2 = [x - bar_width for x in r1]
r3 = [x + bar_width for x in r1]
r4 = [x + bar_width for x in r3]
ind_list = [r1, r3, r4]

plt.grid(alpha = 0.3)
plt.bar(r2, occ_19["total_perc_respondent"].unique()[:8], width = bar_width, color = "grey")
for i in range(len(ind_list)):
    income_group = list(color_dict.keys())[i]
    data = occ_19[occ_19["income_group"] == income_group].iloc[:8, :]
    plt.bar(ind_list[i], data["perc_respondent"], width = bar_width, color = color_dict[income_group])
    
plt.xticks([r for r in range(8)], 
           occ_19["occupation"].unique()[:8], size = 11)
plt.yticks(size = 11)
plt.xlabel("Occupation", size = 12)
plt.ylabel("% of Respondents", size = 12)
plt.title("Top 8 Occupations by Number of Respodents (2019)")
plt.ylim(0, 35) 

# Add legend
temp_markers = [plt.Line2D([0,0],[0,0],color=color, marker='o', linestyle='') for color in ["grey"] + list(color_dict.values())]
plt.legend(temp_markers, ["All countries/regions"] + list(color_dict.keys()), prop={'size': 12}, 
           bbox_to_anchor=(0.9, -0.2), ncol = 4)
plt.show()

---

# 5. Programming Languages

In a blog post, David Robinson from Stack Overflow points out that users from high income countries and non-high income countries differ in technologies used. By analyzing Stack Overflow traffic to questions associated with different programming languages, for example, he found that R was the 15th most visited tag from users in high-income countries, but didn’t make the top 50 tags in the rest of the world [4]. David’s blog post inspires me to examine the popularity of programming languages used by data science practitioners in developing countries.

### Programming Languanges by Popularity

One of the survey questions ask respondents what programming languages they use regularly. Based on the survey result, the following grouped histogram visualizes % of respondents by programming language for all countries and regions as well as for each income group. <span style="background-color: #ffe7ad"> Python, SQL and R are the top three most popular languages reported by survey respondents regardless of the income groups.</span> 
- __Python__: Around 65% of Kaggle respondents use Python on a regular basis, and the percentage point does not vary much by country income group. 
- __R and SQL__: On average, 35% and 25% of Kaggle respondents use SQL and R on a regular basis respectively. Higher income country over-index on SQL and R, while lower middle income country under-index on these two languages.

In [ ]:
# Reformat table
def reformat_mc(question_num, mc_df):
    """
    Convert column values from language name to 1/0 dummy 
    with language name as column name
    
    Input:
    question_num(str):   question number (e.g. "Q18")
    mc_df(dataframe):    multiple choice response dataframe
    
    Output:
    lang_list(list):     list of language name
    lang_df(dataframe):  dataframe with dummy variables (without text response)
    """
    # Find columns relevant to a given question
    col_list = [col for col in mc_df.columns if question_num in col][:-1]
    # Identify language names
    lang_list = []
    for i in range(len(col_list)):
        lang = mc_df[col_list[i]].unique().astype(str)
        lang_list.append(lang[lang != 'nan'][0])
    # Reformat data frame
    lang_df = mc_df[col_list].astype(str)
    lang_df.columns = lang_list
    for col in lang_list:
        lang_df[col] = np.where(lang_df[col] == 'nan', 0, 1)
    return lang_list, lang_df

# Reformat programming langauge questions for 2019 survey
lang_list, lang_df_19 = reformat_mc("Q18", mc_19)
# Join income table
lang_df_19["country/region"] = mc_19["Q3"]
lang_df_19 = lang_df_19.merge(country_code)

# Compute the number of respondents by income and language
lang_19 = lang_df_19.groupby(["income_group"]).sum() \
                    .merge(pd.DataFrame(lang_df_19.groupby("income_group")["country/region"].count()),
                          left_index = True, right_index = True)
lang_19.columns = list(lang_19.columns[:-1]) + ["total_respondent"]

# Compute overall number
lang_19.loc["overall", :] = lang_19.sum()

# Compute perc of respondent
for col in lang_19.columns[:-1]:
    lang_19[col] = lang_19[col] * 100 / lang_19["total_respondent"]
lang_19 = lang_19.drop("total_respondent", axis = 1)
lang_list = list(lang_19.T.sort_values("overall", ascending = False).index)[2:]

# Create grouped histogram
plt.figure(figsize=(15,4))
bar_width = 0.2
r1 = np.arange(len(lang_list))
r2 = [x - bar_width for x in r1]
r3 = [x + bar_width for x in r1]
r4 = [x + bar_width for x in r3]
ind_list = [r1, r3, r4]

plt.grid(alpha = 0.3)
plt.bar(r2, lang_19.loc["overall", lang_list], width = bar_width, color = "grey")
for i in range(len(ind_list)):
    income_group = list(color_dict.keys())[i]
    plt.bar(ind_list[i], lang_19.loc[income_group, lang_list], width = bar_width, color = color_dict[income_group])
    
plt.xticks([r for r in range(len(lang_list))], lang_list, size = 11)
plt.yticks(size = 11)
plt.xlabel("Programming Language", size = 12)
plt.ylabel("% of Respondents", size = 12)
plt.title("% of Respondent by Programming Language (2019)")

# Add legend
temp_markers = [plt.Line2D([0,0],[0,0],color=color, marker='o', linestyle='') for color in ["grey"] + list(color_dict.values())]
plt.legend(temp_markers, ["All countries/regions"] + list(color_dict.keys()), prop={'size': 12}, 
           bbox_to_anchor=(0.9, -0.2), ncol = 4)
plt.show()

Although all income groups agree on the ranking of Python, SQL and R, the following slope chart shows <span style="background-color: #ffe7ad"> a discrepancy among income groups </span>. For example:
- __C++__: C++ is the 7th most popular language in the developed countries, but it is ranked as the 4th most popular language in developing countries. 
- __Bash__: Bash is the 4th most popular language in developing countries, but its rank is 7th in the upper middle income group and 9th in the lower middle income group. 

In [ ]:
# Compare ranking
lang_rank = pd.DataFrame({"index" : np.arange(len(lang_list) + 1, 1, -1)})
for income_group in color_dict.keys():
    lang_rank[income_group] = lang_19[lang_list].T.sort_values(income_group, ascending = False).index

# Drop slope graph
plt.figure(figsize=(14, 5))
ind = [2, 5, 8]
# Loop through income group
for j in range(3):
    plt.text(ind[j]-0.2, 2.8, list(color_dict.keys())[j], size = 14,
            bbox=dict(facecolor= color_dict[list(color_dict.keys())[j]], 
                      boxstyle='round', edgecolor = 'none', alpha = 0.3))
    # Loop through language
    for i in range(len(lang_rank)):
        plt.text(ind[j], lang_rank.iloc[i, 0]/5, "#{} {}".format(str(i+1), lang_rank.iloc[i, j+1]), size = 12)

x1 = [3.15, 4.8]
y1, y2, y3, y4 = [2.05, 1.45], [1.45, 2.05], [1.05, 1.25], [0.85, 1.05]
plt.plot(x1, y1, marker = 'o', color = color_dict["High income"])
plt.plot(x1, y2, x1, y3, x1, y4, marker = 'o', color = color_dict["Upper middle income"])

x2 = [6.15, 7.8]
y5, y6, y7, y8 = [1.85, 1.45], [1.65, 1.85], [1.45, 1.05], [1.05, 1.65]
plt.plot(x2, y5, x2, y7, marker = 'o', color = color_dict["Upper middle income"])
plt.plot(x2, y6, x2, y8, marker = 'o', color = color_dict["Lower middle income"])

plt.title("Most Popular Programming Languages by Income Group", size = 14)
plt.xlim(1, 10)
plt.ylim(0.7, 3)
plt.axis('off')
plt.show()

### Correlation between Popularity and GNI per Capita

The differences shown above further motivate me to examine programming language popularity by countries as David did in his blog post. The following scatterplots illustrate the % of respondents that report to use a given programming la"nguage regularly (y-axis) by the log gross national income per capita (x-axis) for the top eight most popular languages. Interestingly, we can observe some moderate to strong correlation between these two variables:
- __C and C++__: <span style="background-color: #ffe7ad"> % of respondents that use C and C++ are negatively correlated with countries’ GNI perpita </span>. The negative correlation may be related to the fact that <span style="background-color: #ffe7ad"> there are proportionally more students in developing countries, and C and C++ are often used in the introductory programming classes in universities </span>. Anecdotally, I also find that when intro to programming classes in most of the U.S. universities now teach students to use Python, those in developing countries such as China and India still use C and C++ as primary tools. 
- __Python, SQL, R and Bash__: <span style="background-color: #ffe7ad"> % of respondents that use Python, SQL, R and Bash are positively correlated with countries' GNI per capita </span>. In particular, Bash shows the strongest correlation with a coefficient of 0.71. The positive correlation may be related to the fact that there are proportionally more data science professionals in the developed countries. For example, Bash, a Unix shell and command language, are more likely to be used by data science professionals to automate tasks and run scripts on remote clusters. 


In [ ]:
# Compute number of respondent by country and language
res_by_country = lang_df_19.groupby(["country_code", "country/region"])["income_group"].count().reset_index()
res_by_country.columns = ["country_code", "country/region", "total_respondent"]
lang_by_country = lang_df_19.groupby(["income_group", "country_code"])[lang_list].sum().reset_index() \
                            .merge(res_by_country) \
                            .merge(gni_df.dropna(subset = ["gni_per_capita"]))

# Compute perc of respondent by language for each country
for lang in lang_list:
    lang_by_country[lang] = lang_by_country[lang] * 100 / lang_by_country["total_respondent"]

# Create scatterplot
fig, axs = plt.subplots(4, 2, figsize=(15,25))
for i, ax in enumerate(fig.axes):
    lang = lang_list[i]
    ax.grid(alpha = 0.3)
    x = np.log(lang_by_country["gni_per_capita"])
    y = lang_by_country[lang]
    # Plot points
    ax.scatter(x, y,
                c = [color_dict[ig] for ig in lang_by_country["income_group"]],
                s = lang_by_country["total_respondent"]/5,
                alpha = 0.7)
    # Plot fitted line
    b, m = polyfit(x, y, 1)
    ax.plot(x, b + m * x, linestyle = "--", color = "grey", alpha = 0.5)
    ax.set_xlabel("Log GNI per Capita (US Dollar)", size = 10)
    ax.set_ylabel("% of Respondents", size = 10)
    ax.set_title("{} (r = {})".format(lang, str(round(np.corrcoef(x, y)[0][1], 2))), size = 12)
    texts = [ax.annotate(lang_by_country["country_code"][i], (x[i],y[i]), size = 8) for i in range(len(x))]
    adjust_text(texts, ax = ax, only_move={"text":"y"})
    
plt.xticks(size = 11)
plt.yticks(size = 11)
# Add legend for colors
legend1 = plt.legend(markers, color_dict.keys(), prop={'size': 12}, 
              bbox_to_anchor=(0.5, -0.2), ncol = 3)
# Add lengends for size
g1 = plt.scatter([],[], s=20, marker='o', color='#555555', alpha=0.5)
g2 = plt.scatter([],[], s=100, marker='o', color='#555555', alpha=0.5)
g3 = plt.scatter([],[], s=200, marker='o', color='#555555', alpha=0.5)
legend2 = plt.legend((g1, g2, g3), ("100", "500", "1000"), prop={'size': 12},
              bbox_to_anchor=(0.2, -0.4), ncol = 3 , title = "Number of Respondents")
legend2.get_title().set_fontsize('12')
ax.add_artist(legend1)
ax.add_artist(legend2)
plt.show()

---
# 6. Discussion

This notebook focuses on a subset of the data science community, data science practitioners in developing countries, by analyzing the demographic composition as well as the programming languages used by Kaggle survey respondents. This analysis finds that <span style="background-color: #ffe7ad">Kagglers from developing countries are more likely to be data science learners at younger ages </span>. In addition, <span style="background-color: #ffe7ad">the % of respondents who use Python and R, two languages supported by Kaggle, is positively correlated with countries’ income level </span>.

I hope that this notebook can help Kaggle better understand the priorities and needs of data science learners in developing countries, which account for more than half of the survey respondents. To make data science more accessible to a wider range of audiences, Kaggle can consider the following strategies:

- __More entry-level competitions:__ Kaggle can launch more playground or knowledge based competitions to encourage broader participation
- __Multilingual documentation:__ When Kaggle is only available in English, the official languages in most of the developing countries are not English. It would be helpful if the Kaggle Help and Documentation page (https://www.kaggle.com/docs) could be translated into different languages.
- __Collaboration with online courses:__ Many platforms offer online data science courses that are available in different languages and are very popular in developing countries. Kaggle can partner with these online courses and attract more users from developing countries.

---
# 7. Reference
[1] Anthony Goldbloom, "Reviewing 2018 and Previewing 2019", http://blog.kaggle.com/2019/01/18/reviewing-2018-and-previewing-2019/

[2] World Development Indicators, http://datatopics.worldbank.org/world-development-indicators/

[3]  Espen Beer Prydz and Divyanshi Wadhwa, "Classifying countries by income", https://datatopics.worldbank.org/world-development-indicators/stories/the-classification-of-countries-by-income.html

[4] David Robinson, "A Tale of Two Industries: How Programming Languages Differ Between Wealthy and Developing Countries", https://stackoverflow.blog/2017/08/29/tale-two-industries-programming-languages-differ-wealthy-developing-countries/